In [1]:
!pip install fasttext-wheel

In [2]:

import ast
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
import fasttext
import fasttext.util


In [3]:


# Load FastText model
fasttext_model = fasttext.load_model('crawl-300d-2M-subword/crawl-300d-2M-subword.bin')

In [4]:
# Load data and labels
def load_data_labels(data_path, labels_path):
    with open(data_path, 'r') as f:
        data = [ast.literal_eval(line.strip()) for line in f]
    with open(labels_path, 'r') as f:
        labels = [ast.literal_eval(line.strip()) for line in f]
    return data, labels

# Parameters
embedding_dim = 300
output_dim = 3
max_length = 100
batch_size = 1024

# Load training data
data, labels = load_data_labels('training_data_processed.txt', 'order_category_labels.txt')



# Prepare labels with padding
Y = pad_sequences(labels, maxlen=max_length, padding='post', value=2)

In [5]:
# Function to compute embeddings for a batch dynamically
def compute_batch_embeddings(sentences):
    batch_embeddings = []
    for sentence in sentences: 
        sentence_embeddings = [fasttext_model.get_word_vector(word) for word in sentence]
        # Pad or truncate the embeddings to match max_length
        if len(sentence_embeddings) < max_length:
            sentence_embeddings.extend(
                [np.zeros(embedding_dim)] * (max_length - len(sentence_embeddings))
            )
        batch_embeddings.append(sentence_embeddings)
    return np.array(batch_embeddings, dtype=np.float32)

# Updated data generator
def data_generator_dynamic_embeddings(data, labels):
    for sentence, label in zip(data, labels):  # ([sentence], [label])
        sentence_embedding = compute_batch_embeddings([sentence]) 
        yield sentence_embedding[0], label

# Wrap data in a tf.data.Dataset with dynamic embedding calculation
dataset_dynamic = tf.data.Dataset.from_generator(
    lambda: data_generator_dynamic_embeddings(data, Y),
    output_signature=(
        tf.TensorSpec(shape=(max_length, embedding_dim), dtype=tf.float32),
        tf.TensorSpec(shape=(max_length,), dtype=tf.int32)
    )
)

dataset_dynamic = dataset_dynamic.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:

# Model definition
Bidirectional_LSTM_model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.0, input_shape=(max_length, embedding_dim)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(output_dim, activation='softmax')
])

# Compile the model
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-9, 
    verbose=1
)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
Bidirectional_LSTM_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
Bidirectional_LSTM_model.fit(dataset_dynamic, epochs=20, callbacks=[lr_scheduler])

# Model summary
Bidirectional_LSTM_model.summary()

# # # __ddasddasdsadas

Epoch 1/20
1954/1954 [==============================] - 496s 249ms/step - loss: 0.0602 - accuracy: 0.7766 - lr: 0.0010
Epoch 2/20
1954/1954 [==============================] - 478s 244ms/step - loss: 0.0593 - accuracy: 0.7788 - lr: 0.0010
Epoch 3/20
1954/1954 [==============================] - 474s 242ms/step - loss: 0.0570 - accuracy: 0.7836 - lr: 0.0010
Epoch 4/20
1954/1954 [==============================] - 477s 244ms/step - loss: 0.0564 - accuracy: 0.7854 - lr: 0.0010
Epoch 5/20
1954/1954 [==============================] - 480s 246ms/step - loss: 0.0553 - accuracy: 0.7882 - lr: 0.0010
Epoch 6/20
1954/1954 [==============================] - 478s 245ms/step - loss: 0.0545 - accuracy: 0.7900 - lr: 0.0010
Epoch 7/20
1954/1954 [==============================] - 484s 248ms/step - loss: 0.0541 - accuracy: 0.7911 - lr: 0.0010
Epoch 8/20
1954/1954 [==============================] - 480s 245ms/step - loss: 0.0538 - accuracy: 0.7915 - lr: 0.0010
Epoch 9/20
1954/1954 [==========================

In [6]:
Bidirectional_LSTM_model.save('Bidirectional_LSTM_model.h5')

NameError: name 'Bidirectional_LSTM_model' is not defined

In [7]:
model= tf.keras.models.load_model('Bidirectional_LSTM_model.h5')
preds_training = model.predict(dataset_dynamic)



    781/Unknown - 205s 252ms/step

InternalError: Graph execution error:

Detected at node 'cond/CudnnRNNV3' defined at (most recent call last):
    File "c:\Users\DELL\anaconda3\envs\NLP\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\DELL\anaconda3\envs\NLP\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\DELL\anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\DELL\anaconda3\envs\NLP\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\DELL\anaconda3\envs\NLP\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\DELL\anaconda3\envs\NLP\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\DELL\anaconda3\envs\NLP\lib\asyncio\base_events.py", line 57

In [ ]:
preds_training = np.argmax(preds_training, axis=-1)
correct_predictions = 0
total_sentences = len(data)

for i in range(total_sentences):
    # Get the unpadded length of the current sentence
    unpadded_length = len(data[i])
    
    # Slice the predictions and labels up to the unpadded length
    pred = preds_training[i][:unpadded_length]
    true = labels[i][:unpadded_length]
    
    # Check if the predictions match the ground truth
    if (pred == true).all():
        correct_predictions += 1
    
    # (Optional) Debugging
    print(f"Sentence {i + 1}/{total_sentences}")
    print(f"Prediction: {pred}")
    print(f"Ground Truth: {true}")
    print("-----------------------------------")

# Compute accuracy
accuracy = correct_predictions / total_sentences
print(f"Accuracy on dev data: {accuracy:.4f}")

In [ ]:
dev_data, dev_labels = load_data_labels('dev_data_processed.txt', 'dev_order_category_labels.txt')
Y_dev=pad_sequences(dev_labels, maxlen=max_length, padding='post', value=2)
dev_dynamic = tf.data.Dataset.from_generator(
    lambda: data_generator_dynamic_embeddings(dev_data, Y_dev),
    output_signature=(
        tf.TensorSpec(shape=(max_length, embedding_dim), dtype=tf.float32),
        tf.TensorSpec(shape=(max_length,), dtype=tf.int32)
    )
)

dev_dynamic = dev_dynamic.batch(batch_size).prefetch(tf.data.AUTOTUNE)


preds_dev = Bidirectional_LSTM_model.predict(dev_dynamic)
preds_dev = np.argmax(preds_dev, axis=-1)


1/1 [==============================] - 0s 253ms/step
[[0 0 2 0 0 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 2 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
correct_predictions = 0
total_sentences = len(dev_data)

for i in range(total_sentences):
    # Get the unpadded length of the current sentence
    unpadded_length = len(dev_data[i])
    
    # Slice the predictions and labels up to the unpadded length
    pred = preds_dev[i][:unpadded_length]
    true = dev_labels[i][:unpadded_length]
    
    # Check if the predictions match the ground truth
    if (pred == true).all():
        correct_predictions += 1
    
    # (Optional) Debugging
    print(f"Sentence {i + 1}/{total_sentences}")
    print(f"Prediction: {pred}")
    print(f"Ground Truth: {true}")
    print("-----------------------------------")

# Compute accuracy
accuracy = correct_predictions / total_sentences
print(f"Accuracy on dev data: {accuracy:.4f}")


Sentence 1/348
Prediction: [0 0 0 0 0 2 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0]
Ground Truth: [2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
-----------------------------------
Sentence 2/348
Prediction: [0 0 0 0 0 0 0]
Ground Truth: [0, 0, 0, 0, 0, 0, 0]
-----------------------------------
Sentence 3/348
Prediction: [0 1 0 2 0 0 0 0 0 2 2 2]
Ground Truth: [2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]
-----------------------------------
Sentence 4/348
Prediction: [0 0 2 0 0 2 2 2 2 2]
Ground Truth: [2, 2, 2, 2, 0, 0, 0, 0, 0, 0]
-----------------------------------
Sentence 5/348
Prediction: [2 0 2 0 0 0 0 0 0 0 0 2 2]
Ground Truth: [2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-----------------------------------
Sentence 6/348
Prediction: [0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 1]
Ground Truth: [2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1]
-----------------------------------
Sentence 7/348
Prediction: [2 2 2 0 0 0 